# Deep Learning Overview


---

### 📦 1. Install Required Packages


In [ ]:
!pip install --upgrade tensorflow numpy


### 2.Import necessary Libs

In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 3. Dataset Preparation Function 

In [ ]:
tokenizer = Tokenizer()

In [ ]:

def dataset_preparation(data):

    corpus = data.lower().split("\n")

    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            input_sequences.append(token_list[:i + 1])

    max_sequence_len = max(len(seq) for seq in input_sequences)

    input_sequences = np.array(
        pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
    )

    predictors = input_sequences[:, :-1]
    labels = input_sequences[:, -1]

    labels = to_categorical(labels, num_classes=total_words)

    return predictors, labels, max_sequence_len, total_words


### 4. Model Creation Function

In [ ]:
def create_model(predictors, labels, max_sequence_len, total_words):

    model = Sequential()
    model.add(Embedding(total_words, 100, input_length=max_sequence_len - 1))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(100))
    model.add(Dense(total_words, activation='softmax'))

    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )

    earlystop = EarlyStopping(
        monitor='loss',
        patience=5,
        restore_best_weights=True
    )

    model.fit(
        predictors,
        labels,
        epochs=100,
        verbose=1,
        callbacks=[earlystop]
    )

    model.summary()
    return model


### ✍️ 5. Text Generation Function (Optimized)

In [ ]:
def generate_text(seed_text, next_words, max_sequence_len, model):

    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences(
            [token_list],
            maxlen=max_sequence_len - 1,
            padding='pre'
        )

        predicted = np.argmax(model.predict(token_list, verbose=0))
        output_word = tokenizer.index_word.get(predicted, "")

        seed_text += " " + output_word

    return seed_text


6. Load Data from Google Drive



1. **Prepare your text file:**
   - Create a `.txt` file with your own text (poems, stories, quotes, etc.)
   - The more text you have, the better the model will learn
   - Minimum recommended: 50-100 lines of text

2. **Upload to Google Drive:**
   - Upload your text file to your Google Drive
   - Note the path to your file

3. **Modify the code:**
   - Update the file path in the data loading cell
   - Change: `data = open('/content/drive/MyDrive/YOUR_PATH/YOUR_FILE.txt').read()`


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data = open('/content/drive/MyDrive/Agentic-AIOT-Workshop/sample1.txt').read()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 7. Train & Generate Text


4. **Train and generate:**
   - Run the training cells again
   - Try different seed texts related to your content
   - Experiment with the `next_words` parameter (try 5, 10, 15, 20)


In [ ]:

predictors, labels, max_sequence_len, total_words = dataset_preparation(data)

model = create_model(
    predictors,
    labels,
    max_sequence_len,
    total_words
)

print(generate_text("it can process", 5, max_sequence_len, model))
print("--Over--")


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 104ms/step - accuracy: 0.0181 - loss: 4.2060
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.0530 - loss: 4.1944
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.0608 - loss: 4.1684
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - accuracy: 0.0349 - loss: 4.1141
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.0569 - loss: 4.1053
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.0530 - loss: 4.0238
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.0814 - loss: 3.9198
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.1099 - loss: 3.8218
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 178ms/step - accuracy: 0.1074 - loss: 3.7633
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 182ms/step - accuracy: 0.0775 - loss: 3.7237
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 180ms/step - accuracy: 0.0775 - loss: 3.6281
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 187ms/step - accuracy: 

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 28, 100)        │         6,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 28, 150)        │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 28, 100)        │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 100)            │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 67)             │         6,767 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,034,603 (3.95 MB)

 Trainable params: 344,867 (1.32 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 689,736 (2.63 MB)

it can process lstm only single data such
--Over--


### Generate text


4. **Train and generate:**
   - Run the training cells again
   - Try different seed texts related to your content
   - Experiment with the `next_words` parameter (try 5, 10, 15, 20)


**Challenge:** Try training on different text types and compare the results!


In [ ]:
print(
    generate_text(
        seed_text="lstm is",
        next_words=10,
        max_sequence_len=max_sequence_len,
        model=model
    )
)


lstm is lstm lstm feedback connections such as unsegmented connected handwriting recognition


In [ ]:
print(generate_text( seed_text="deep learning",
        next_words=5,max_sequence_len=max_sequence_len,model=model)
)

deep learning lstm lstm is to tasks



## Learning Resources

### 📘 Neural Networks (Foundations)

- Neural Networks Explained (YouTube)  
  https://youtu.be/alfdI7S6wCY?si=PCvbro1kgdfPPjzn

---

### 📘 Recurrent Neural Networks (RNN)

- Text Generation using RNN (Interactive Tutorial)  
  https://artinte.github.io/deep-learning/text_generate_rnn.html

---

### 📘 LSTM

- Understanding LSTM Networks (Colah’s Blog)  
  https://colah.github.io/posts/2015-08-Understanding-LSTMs/

- Gentle Introduction to LSTM  
  https://machinelearningmastery.com/gentle-introduction-long-short-term-memory-networks-experts/

---

### 📘 Deep Learning References

- Deep Learning Book – RNN Chapter  
  https://www.deeplearningbook.org/contents/rnn.html

- TensorFlow RNN Guide  
  https://www.tensorflow.org/guide/keras/rnn

---

### 📘 Practical Tutorials

- Text Generation with TensorFlow  
  https://www.tensorflow.org/tutorials/text/text_generation

- Keras LSTM API Documentation  
  https://keras.io/api/layers/recurrent_layers/lstm/

---

### 📘 Research Papers

- Original LSTM Paper (Hochreiter & Schmidhuber)  
  https://www.bioinf.jku.at/publications/older/2604.pdf

- Empirical Evaluation of Gated RNNs  
  https://arxiv.org/abs/1409.2329
